In [3]:
import os
import xml.etree.ElementTree as ET

In [1]:
import pandas as pd
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

ModuleNotFoundError: No module named 'google'

In [12]:

CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
# Remove keyword arguments that are not set
def remove_empty_kwargs(**kwargs):
    good_kwargs = {}
    if kwargs is not None:
        for key, value in kwargs.items():
            if value:
                good_kwargs[key] = value
    return good_kwargs

In [15]:
client = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=692697523900-m6jnb73lek6udl104gvou0joh8t0771q.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=O2rgDFrfJ5kVY062gbALmtf5oW3yKz&prompt=consent&access_type=offline
Enter the authorization code: 4/0AEtq3xJseFOZUi2wEx63TZEGh4StR31UgrWxjuRbZDDaHq9cMuUe7Y


In [16]:
def youtube_keyword(client, **kwargs):    
    kwargs = remove_empty_kwargs(**kwargs)
    response = client.search().list(
        **kwargs
        ).execute()    
    return response

In [35]:
def youtube_search (criteria,max_res):   
    #create lists and empty dataframe
    titles = []
    videoIds = []
    channelIds = []
    resp_df = pd.DataFrame()
    
    while len(titles) < max_res:
        token = None
        response = youtube_keyword(client,
                        part='id,snippet',
                        maxResults=50,
                        q=criteria,
                        videoCaption='closedCaption',
                        type='video', 
                        #videoDuration='long',
                        #I18nRegion='de',
                        pageToken=token,
                        relevanceLanguage='de',
                        regionCode='de') 
                                         
        for item in response['items']:        
            titles.append(item['snippet']['title'])
            channelIds.append(item['snippet']['channelTitle'])
            videoIds.append(item['id']['videoId'])
        
        token = response['nextPageToken']
        
    resp_df['title'] = titles
    resp_df['channelId'] = channelIds
    resp_df['videoId'] = videoIds
    resp_df['subject'] = criteria
    
    return resp_df

In [37]:
Gates_Videos = youtube_search('[Impfen+Bill+Gates]',10)

HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/search?part=id%2Csnippet&maxResults=50&q=%5BImpfen%2BBill%2BGates%5D&videoCaption=closedCaption&type=video&relevanceLanguage=de&regionCode=de&alt=json returned "Daily Limit Exceeded. The quota will be reset at midnight Pacific Time (PT). You may monitor your quota usage and adjust limits in the API Console: https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=692697523900">

In [7]:
Gates_Videos.loc[0:40]

,Unnamed: 0,title,channelId,videoId,subject
0,0,Bill Gates will die Welt impfen,veggietation,AhuU_CG9v80,[Impfen+Gates]
1,1,Bill Gates und Bevölkerungskontrolle,veggietation,YWjkYM8slFU,[Impfen+Gates]
2,2,Wie Bill Gates das Weltgesundheitssystem monop...,veggietation,QVYj1HQyPg0,[Impfen+Gates]
3,3,Innovating to zero! | Bill Gates,TED,JaF-fq2Zn7I,[Impfen+Gates]
4,4,Bill Gates on Fighting Coronavirus | The Daily...,The Daily Show with Trevor Noah,iyFT8qXcOrM,[Impfen+Gates]
5,5,Dok 1 - &quot;Impfen? Nein danke!&quot;,Christian Tieber,8nH_0sauWxk,[Impfen+Gates]
6,6,Bill Gates at MIT 2010 - &quot;Giving Back: Fi...,MIT Video Productions,bfIqbOi9RCQ,[Impfen+Gates]
7,7,Jens Spahn im Interview mit YouTubern bei Brea...,Bundesministerium für Gesundheit,RsP9AuKKu20,[Impfen+Gates]
8,8,Online-Bürgersprechstunde mit Ministerpräsiden...,FreistaatSachsen,O8iMjdBFY3A,[Impfen+Gates]
9,9,Corona: die krassesten Thesen und Lügen | STRG_F,STRG_F,3duErFbfFM0,[Impfen+Gates]


In [20]:
def get_all_ccs(videos):
    base_url = 'https://www.youtube.com/watch?v='
    #lang="de"
    for vid in videos:
        url = base_url + vid
        cmd = ["youtube-dl","--skip-download","--write-sub",url]
        os.system(" ".join(cmd))

In [7]:
len(set(vids))

537

#### Using the above created function

In [ ]:
get_all_ccs(Gates_Videos.videoId)

In [15]:
sub_vids

['sYT0A26wsxQ', '72plsUzUjP4']

In [4]:
vids = [item[0] for item in Gates_Videos.values.tolist()]

In [5]:
filenames_vtt = [os.fsdecode(file) for file in os.listdir(os.getcwd()) if os.fsdecode(file).endswith(".vtt")]
#Check file names
filenames_vtt

['Pressebriefing der Sächsischen Staatsregierung – aktueller Stand zum Coronavirus in Sachsen 14.05.20-V0J94vF_rZ4.de.vtt',
 'Has COVID-19 Reached The BRAIN-ZIXznkrltwE.en.vtt',
 'Halle (Saale) - Video-Pressekonferenz vom 19. April 2020 zur Corona-Lage – Mit Untertiteln-o0scbzPHFtM.de.vtt',
 'Bill Gates und  Bevölkerungskontrolle-YWjkYM8slFU.de.vtt',
 'Gebärdensprachvideo - AfD-Anträge zu Grund\xadrechten stoßen auf Widerspruch-joqd-Mvpa6E.de.vtt',
 'Jens Spahn im Interview mit YouTubern bei Breaking Lab (Wir gegen Corona)-RsP9AuKKu20.de-DE.vtt',
 "'Pandemie'! Eine schwache Saison Grippe!-0uFYpyfYaXQ.en.vtt",
 'Coronavirus-Update #33 - Herdenimmunität noch lange nicht in Sicht _ NDR Podcast-DSeZYVYd_DU.de.vtt',
 'CORONAVIRUS 😷 2019-nCoV 🤧 ¦¦ Wir sind der VIRUS und vernichten die WELT ★ [ᴴᴰ60ᶠᵖˢ]-6RRYqN5hbkw.en.vtt',
 'Tamina in Thailand _ WDR Reisen-in2dRwiShWY.en.vtt',
 'Extra 3 vom 07.05.2020 mit Christian Ehring im Ersten _ extra 3 _ NDR-8s3hU3YDh3A.de.vtt',
 'How we must respond to

#### Converting vtt to csv

In [6]:
import webvtt
def convert_vtt(filenames):    
    #create an assets folder if one does not yet exist
    if os.path.isdir('{}/assets'.format(os.getcwd())) == False:
        os.makedirs('assets')
    #extract the text and times from the vtt file
    for file in filenames:
        captions = webvtt.read(file)
        text_time = pd.DataFrame()
        text_time['text'] = [caption.text for caption in captions]
        text_time['start'] = [caption.start for caption in captions]
        text_time['stop'] = [caption.end for caption in captions]
        text_time.to_csv('assets/{}.csv'.format(file[:-4]),index=False) #-4 to remove '.vtt'
        #remove files from local drive
        os.remove(file)

In [7]:
#call the function
convert_vtt(filenames_vtt)

In [10]:
csv_files = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets') if os.fsdecode(file).endswith('.csv')]

In [11]:
path = 'assets/'
for filename in csv_files:
    os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '')))
#to verify the results
clean_csv = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets')]
clean_csv[0]

'PoetrySlammer-MATZEB.atTEDxBodensee-KG9tm2Z4W-Y.en.csv'

In [12]:
#extract the text and videoid
vidText = []
csv_vidid = []

for file in clean_csv:
   df = pd.read_csv(path+file)
   text = " ".join(df.text)
   vidText.append(text)
   csv_vidid.append(file[-18:-7])

In [15]:
vtxt = pd.DataFrame(vidText)
vID = pd.DataFrame(csv_vidid)

In [17]:
vtxt.to_csv('vidTexts.csv', index = False)
vID.to_csv('vidID.csv', index = False)

In [58]:
vidText[12]


"of of of of this of of of of of of of of house of blues of of\nof a of of of of of of the of an aunt of force and not make the the same as the end of the facts of this as a foundation of Santa has fashioned a variant of that, and he has this have to have come to the forefront ruled no\nplace for common sense to the\nsuns the info as 2020 bodies might dish out which can only mean one thing programme you are watching the Conejo,\nwhich discusses policies social\nissues my name is my de my name is my de chat through the prism of common\nsense my name is my de Charles my name is mine de chacal my name is my name is mine de chacal de Challes de Challes you are de Challes you are we de Challes you are one de Challes you are watch de Challes you are we see take it with us in the de Challes you are watching on to resets de Challes you are watching on\nparle de prison what it all these de Challes you are watching on\nparle de prison oui the crusty old office says you\nknow de Challes you are w

In [50]:
Gates_Videos.to_csv('GatesVids.csv')

In [3]:
Gates_Videos = pd.read_csv("GatesVids.csv")

In [4]:
Gates_Videos = pd.read_csv("idlst.csv")